In [1]:
####!/usr/bin/python
##### coding: utf-8

In [2]:
#Action Plan

#1-Get and prepare data
#2-Normalize Data
#3-Processing Data
#4-Prepare Data to DDBB
#5-Upload data to DDBB

In [60]:
#Import required libraries

import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import normalize
import psycopg2, psycopg2.extras
import psycopg2.extensions
import sys
psycopg2.extensions.register_type(psycopg2.extensions.UNICODE) 
psycopg2.extensions.register_type(psycopg2.extensions.UNICODEARRAY)

#See al columns
#pd.set_option('display.max_columns',50).


### 1-Get Data

In [108]:
#Connect to database
conn = psycopg2.connect(database='cityData', host='localhost',user = 'postgres', password="DS17")
cur = conn.cursor() 

In [76]:
#Create Table for results

cur.execute("CREATE TABLE seccresults (idsecc BIGINT  PRIMARY KEY,idseccmad BIGINT)")

#Send Commit

conn.commit()

#### Data from Censo 2011

In [46]:
#%%time
#Select all data except Madrid city data
cur.execute("SELECT * FROM secc,seccdetail  where  seccdetail.idsecc=secc.idsecc and seccdetail.idccaa <> 13 and seccdetail.codmun <> 79 ")
ver = cur.fetchall()
indexed_df_secc = pd.DataFrame(ver).set_index([0])
indexed_df_secc = indexed_df_secc.iloc[:,:145]
indexed_df_secc.head()

,1,2,3,4,5,6,7,8,9,10,...,136,137,138,139,140,141,142,143,144,145
0,,,,,,,,,,,,,,,,,,,,,
10400101001,1460,760,695,170,930,360,1300,155,0,5,...,90,80,100,595,140,215,120,85,35,0
10400201001,1350,700,655,150,870,335,1270,80,0,0,...,135,50,50,550,125,170,145,90,15,0
10400301001,765,325,445,70,590,105,740,0,0,0,...,0,0,0,295,0,80,0,80,0,0
10400301002,1750,800,950,335,1140,275,1695,0,45,0,...,0,0,0,650,105,230,105,175,30,0
10400301003,1815,780,1035,470,1120,225,1660,0,10,145,...,0,0,0,610,50,170,180,170,20,15


In [47]:
#Normalize Data
X_normalized_indexed_df_secc = preprocessing.normalize(indexed_df_secc, norm='l1')
X_normalizeddf_seccL2 = preprocessing.normalize(indexed_df_secc, norm='l2')
ix1 =indexed_df_secc.index
col1 = indexed_df_secc.columns
df__normalized_indexed_secc=pd.DataFrame(X_normalized_indexed_df_secc, index=[ix1] ,columns=[col1])
df__normalized_indexed_secc.head()

,1,2,3,4,5,6,7,8,9,10,...,136,137,138,139,140,141,142,143,144,145
0,,,,,,,,,,,,,,,,,,,,,
10400101001,0.053795,0.028003,0.025608,0.006264,0.034267,0.013265,0.047900,0.005711,0.000000,0.000184,...,0.003316,0.002948,0.003685,0.021923,0.005158,0.007922,0.004422,0.003132,0.001290,0.000000
10400201001,0.055158,0.028601,0.026762,0.006129,0.035546,0.013687,0.051890,0.003269,0.000000,0.000000,...,0.005516,0.002043,0.002043,0.022472,0.005107,0.006946,0.005924,0.003677,0.000613,0.000000
10400301001,0.057977,0.024631,0.033725,0.005305,0.044714,0.007958,0.056082,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.022357,0.000000,0.006063,0.000000,0.006063,0.000000,0.000000
10400301002,0.057227,0.026161,0.031066,0.010955,0.037279,0.008993,0.055428,0.000000,0.001472,0.000000,...,0.000000,0.000000,0.000000,0.021256,0.003434,0.007521,0.003434,0.005723,0.000981,0.000000
10400301003,0.056192,0.024149,0.032043,0.014551,0.034675,0.006966,0.051393,0.000000,0.000310,0.004489,...,0.000000,0.000000,0.000000,0.018885,0.001548,0.005263,0.005573,0.005263,0.000619,0.000464


In [48]:
#%time
#Select data from Madrid
cur.execute("SELECT * FROM secc,seccdetail  where  seccdetail.idsecc=secc.idsecc and seccdetail.idccaa = 13 and seccdetail.codmun = 79 ")
ver =  cur.fetchall()
indexed_df_secc_mad = pd.DataFrame(ver).set_index([0])
indexed_df_secc_mad = indexed_df_secc_mad.iloc[:,:145]
indexed_df_secc_mad.head()



,1,2,3,4,5,6,7,8,9,10,...,136,137,138,139,140,141,142,143,144,145
0,,,,,,,,,,,,,,,,,,,,,
132807901001,1205,480,725,150,700,355,955,50,0,0,...,0,0,75,490,170,160,65,50,0,50
132807901002,855,355,505,75,665,115,720,0,0,0,...,0,0,0,485,195,240,0,0,0,0
132807901003,1795,880,920,130,1350,320,1295,110,0,10,...,25,0,65,920,380,320,115,110,0,0
132807901004,1070,495,575,130,785,150,840,0,0,0,...,0,0,0,550,300,70,130,30,0,20
132807901006,1950,895,1055,200,1305,440,1800,35,0,0,...,10,70,25,960,455,210,180,70,20,30


In [49]:
#Normalize Data
X_normalized_indexed_df_mad = preprocessing.normalize(indexed_df_secc_mad, norm='l1')
X_normalizeddf_seccL2 = preprocessing.normalize(indexed_df_secc_mad, norm='l2')
ix =indexed_df_secc_mad.index
col = indexed_df_secc_mad.columns
df__normalized_indexed_mad=pd.DataFrame(X_normalized_indexed_df_mad, index=[ix] ,columns=[col])
df__normalized_indexed_mad.tail()

,1,2,3,4,5,6,7,8,9,10,...,136,137,138,139,140,141,142,143,144,145
0,,,,,,,,,,,,,,,,,,,,,
132807921026,0.056933,0.028306,0.028626,0.010395,0.042540,0.003998,0.044938,0.001279,0.000640,0.00032,...,0.000000,0.000000,0.000000,0.021110,0.002559,0.009435,0.002879,0.004958,0.000480,0.000640
132807921027,0.056758,0.029227,0.027431,0.014963,0.038504,0.003192,0.052070,0.001895,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.020549,0.003691,0.005287,0.004489,0.006384,0.000698,0.000000
132807921028,0.055092,0.026549,0.028543,0.011689,0.040413,0.002990,0.049112,0.003625,0.000181,0.00000,...,0.001631,0.000816,0.000362,0.023831,0.007521,0.007521,0.003896,0.004078,0.000362,0.000453
132807921029,0.058572,0.028635,0.029937,0.021569,0.034771,0.002417,0.055969,0.001673,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.018036,0.001859,0.003533,0.004463,0.005020,0.003347,0.000000
132807921030,0.055866,0.025372,0.030261,0.014898,0.035847,0.005121,0.049348,0.001164,0.000466,0.00000,...,0.000000,0.000000,0.000000,0.021648,0.004888,0.005587,0.005354,0.004888,0.000466,0.000466


## <p style="text-align: center;">Processing data </p>
<p style="text-align: center;">We use cosine similarity to find the distances between vectors.</p>
$$sim (A,B) = cos(\phi)= \frac{A \cdot B}{||A|| \cdot ||B||}$$

In [10]:
#Processing data 

#Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
func =cosine_similarity(df__normalized_indexed_mad,df__normalized_indexed_secc)
dfunc = pd.DataFrame(func, index=[ix],columns=[ix1])
dfunc.head()

,10400101001,10400201001,10400301001,10400301002,10400301003,10400301004,10400301005,10400301006,10400301007,10400302001,...,195200108005,195200108006,195200108007,195200108008,195200108009,195200108010,195200108011,195200108012,195200108013,195200108014
0,,,,,,,,,,,,,,,,,,,,,
132807901001,0.934543,0.924619,0.920835,0.929042,0.935487,0.912305,0.932546,0.943995,0.936364,0.907335,...,0.916564,0.925027,0.950778,0.925131,0.924339,0.940986,0.925446,0.918390,0.942421,0.909524
132807901002,0.913229,0.911719,0.919057,0.917211,0.908282,0.911169,0.911315,0.936162,0.924441,0.905099,...,0.921695,0.917598,0.941159,0.918170,0.913000,0.939633,0.919235,0.898468,0.957279,0.909126
132807901003,0.884210,0.871883,0.869689,0.879529,0.877139,0.874581,0.881262,0.903635,0.885500,0.878117,...,0.881960,0.880114,0.924074,0.886789,0.877228,0.903525,0.897447,0.880922,0.932803,0.859487
132807901004,0.929373,0.923661,0.924825,0.933692,0.932181,0.925989,0.931921,0.951327,0.932145,0.924679,...,0.931130,0.930415,0.961439,0.933414,0.925305,0.952669,0.944083,0.933993,0.962687,0.913605
132807901006,0.953319,0.956426,0.952753,0.960700,0.951667,0.947766,0.956929,0.971046,0.958877,0.939831,...,0.959909,0.963854,0.975012,0.961946,0.950352,0.976015,0.954018,0.942500,0.976018,0.952201


In [109]:
#Select the maximum value
df =pd.DataFrame(dfunc.idxmax())
df.index.names = ['Secc']
df.shape


(31450, 1)

In [82]:
%%time

#Divided into three parts to avoid memory problems

df= df[:15000] 
#Prepare SQL string
def check_sql_string(sql, a):
    unique = "%PARAMETER%"*len(df)
    sql = sql.replace("?", unique)
    for v in a: sql = str(sql.replace(unique, repr(v), 1))
    return sql
a=df.to_records().tolist()
rw= ', ?'*(len(df)-1)+';'
q = """INSERT INTO seccresults (idsecc,idseccmad) VALUES ?""" +rw 

#test SQL
#print (check_sql_string(q,a))

#Upload Data
cur.execute(check_sql_string(q,a))
conn.commit()

CPU times: user 1h 5min 6s, sys: 1h 26min 47s, total: 2h 31min 54s
Wall time: 2h 32min 51s


In [106]:
%%time

#Divided into three parts to avoid memory problems

df= df[15000:25000] 
#Prepare SQL string
def check_sql_string(sql, a):
    unique = "%PARAMETER%"*len(df)
    sql = sql.replace("?", unique)
    for v in a: sql = str(sql.replace(unique, repr(v), 1))
    return sql
a=df.to_records().tolist()
rw= ', ?'*(len(df)-1)+';'
q = """INSERT INTO seccresults (idsecc,idseccmad) VALUES ?""" +rw 

#test SQL
#print (check_sql_string(q,a))

#Upload Data
cur.execute(check_sql_string(q,a))
conn.commit()

CPU times: user 19min 24s, sys: 25min 37s, total: 45min 1s
Wall time: 45min 16s


In [110]:
%%time

#Divided into three parts to avoid memory problems

df= df[25000:] 
#Prepare SQL string
def check_sql_string(sql, a):
    unique = "%PARAMETER%"*len(df)
    sql = sql.replace("?", unique)
    for v in a: sql = str(sql.replace(unique, repr(v), 1))
    return sql
a=df.to_records().tolist()
rw= ', ?'*(len(df)-1)+';'
q = """INSERT INTO seccresults (idsecc,idseccmad) VALUES ?""" +rw 

#test SQL
#print (check_sql_string(q,a))

#Upload Data
cur.execute(check_sql_string(q,a))
conn.commit()

CPU times: user 5min 11s, sys: 6min 49s, total: 12min
Wall time: 12min 4s


In [83]:
from sklearn.metrics.pairwise import euclidean_distances
X = [[0, 1], [1, 1]]
# distance between rows of X
#euclidean_distances(X, X)


# get distance to origin
euclidf = pd.DataFrame(euclidean_distances(df__normalized_indexed_secc,df__normalized_indexed_mad), index=[ix1],columns=[ix])
eudforder = dfunc.sort_values(dfunc.columns[0],ascending=False)
eudforder[0:5]

Secc,10400101001,10400201001,10400301001,10400301002,10400301003,10400301004,10400301005,10400301006,10400301007,10400302001,...,195200108005,195200108006,195200108007,195200108008,195200108009,195200108010,195200108011,195200108012,195200108013,195200108014
0,,,,,,,,,,,,,,,,,,,,,
132807910063,0.977660,0.981190,0.971154,0.976476,0.970433,0.963776,0.976478,0.974344,0.969061,0.949176,...,0.971295,0.975926,0.951666,0.967032,0.963542,0.966096,0.933053,0.947184,0.961701,0.972581
132807908027,0.976795,0.977734,0.972292,0.976555,0.970288,0.962032,0.971937,0.977894,0.974526,0.958055,...,0.965715,0.974962,0.965055,0.969351,0.964354,0.973218,0.949343,0.954314,0.967613,0.967567
132807910151,0.976767,0.970317,0.951209,0.955737,0.953596,0.941287,0.961766,0.960340,0.956213,0.929496,...,0.949098,0.949967,0.942708,0.941459,0.937403,0.949495,0.916768,0.923499,0.950240,0.940930
132807911006,0.976430,0.981295,0.962938,0.977735,0.970293,0.958965,0.975398,0.970825,0.967273,0.952865,...,0.970174,0.976037,0.969916,0.965617,0.960709,0.970600,0.949836,0.958399,0.961911,0.964935
132807911152,0.976245,0.983607,0.968413,0.987300,0.979508,0.963364,0.979003,0.975135,0.971096,0.958037,...,0.973590,0.984647,0.975472,0.980605,0.971029,0.976022,0.964409,0.968694,0.954823,0.969815


In [ ]:
from sklearn.metrics.pairwise import manhattan_distances
manhattandf = pd.DataFrame(manhattan_distances(X_normalized_indexed_df_mad,df__normalized_indexed01), index=[ix],columns=[ix01])
mandforder = dfunc.sort_values(dfunc.columns[0],ascending=False)
mandforder[0:5]

In [111]:
indexed_df_secc_mad.shape

(2409, 145)

In [112]:
indexed_df_secc.index

Int64Index([ 10400101001,  10400201001,  10400301001,  10400301002,
             10400301003,  10400301004,  10400301005,  10400301006,
             10400301007,  10400302001,
            ...
            195200108005, 195200108006, 195200108007, 195200108008,
            195200108009, 195200108010, 195200108011, 195200108012,
            195200108013, 195200108014],
           dtype='int64', name=0, length=31450)

In [ ]:
#Processing data 
#Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
func =cosine_similarity(df__normalized_indexed_secc,df__normalized_indexed_mad)
dfunc = pd.DataFrame(func, index=[ix1],columns=[ix])
dfunc.head()

In [44]:
dfunc.sort_values(dfunc.columns[1],ascending=False).head()

Secc,10400101001,10400201001,10400301001,10400301002,10400301003,10400301004,10400301005,10400301006,10400301007,10400302001,...,195200108005,195200108006,195200108007,195200108008,195200108009,195200108010,195200108011,195200108012,195200108013,195200108014
0,,,,,,,,,,,,,,,,,,,,,
132807913160,0.973206,0.987500,0.985998,0.989524,0.973632,0.975723,0.980433,0.983603,0.985038,0.968150,...,0.979485,0.990880,0.971881,0.982823,0.963858,0.982950,0.961691,0.958614,0.969371,0.986082
132807917017,0.975286,0.987099,0.964256,0.975663,0.960997,0.956997,0.968179,0.963083,0.968473,0.958410,...,0.966695,0.974932,0.958680,0.963879,0.951027,0.962057,0.947504,0.952716,0.950598,0.963684
132807916073,0.974954,0.986726,0.968436,0.979816,0.964266,0.963694,0.973089,0.969347,0.973438,0.958632,...,0.973673,0.979975,0.956976,0.969957,0.958234,0.966149,0.940221,0.952094,0.957252,0.974960
132807911128,0.967585,0.986387,0.975644,0.985442,0.968354,0.973062,0.978148,0.973246,0.980770,0.966557,...,0.973444,0.985648,0.958871,0.974847,0.955738,0.970762,0.947962,0.952399,0.950326,0.981475
132807911157,0.972812,0.986379,0.982712,0.988387,0.974462,0.980262,0.984177,0.983942,0.987562,0.979386,...,0.981409,0.987526,0.973499,0.980506,0.959367,0.985863,0.964183,0.964026,0.971317,0.981816


In [42]:
c=dfunc.columns
c
#dfunc.nlargest(3,c)

Secc,10400101001,10400201001,10400301001,10400301002,10400301003,10400301004,10400301005,10400301006,10400301007,10400302001,...,195200108005,195200108006,195200108007,195200108008,195200108009,195200108010,195200108011,195200108012,195200108013,195200108014
0,,,,,,,,,,,,,,,,,,,,,
132807910063,0.977660,0.981190,0.971154,0.976476,0.970433,0.963776,0.976478,0.974344,0.969061,0.949176,...,0.971295,0.975926,0.951666,0.967032,0.963542,0.966096,0.933053,0.947184,0.961701,0.972581
132807908027,0.976795,0.977734,0.972292,0.976555,0.970288,0.962032,0.971937,0.977894,0.974526,0.958055,...,0.965715,0.974962,0.965055,0.969351,0.964354,0.973218,0.949343,0.954314,0.967613,0.967567
132807910151,0.976767,0.970317,0.951209,0.955737,0.953596,0.941287,0.961766,0.960340,0.956213,0.929496,...,0.949098,0.949967,0.942708,0.941459,0.937403,0.949495,0.916768,0.923499,0.950240,0.940930


In [ ]:
dforder = dfunc.sort_values(dfunc.columns[1],ascending=False)
ind = dforder.iloc[0:0, 0:1].columns
top5 = dforder.iloc[0:1,0:0].index
top5v = dforder.iloc[0:1,0:1].values
dict = {}
#dict[top5] = [ top5v]
print ind,top5, top5v, 
dforder
#ind
#results = pd.DataFrame(top5, index = ind)
#results


In [ ]:
df = dfunc.sort_values
df

In [ ]:
pdr= dfunc.iloc[:1,:5]
dfunc.to_csv('out.csv', sep=',')


In [ ]:
url = 'out.csv'
pdr=pd.read_csv(url, sep= ',',header = 0)
pdr

In [ ]:
dforder.head()

In [ ]:
dfunc= dfunc.iloc[:,:6]
a=dfunc.to_records().tolist()
b= dfunc.columns
#dfresults = pd.DataFrame(a, columns=b)
#dfresults = dfresults.set_index(0)
#dfresults = dfresults.rename(columns={'10400101001', '10400201001', '10400301001', '10400301002', '10400301003'}, inplace=True)
a
b
#dfresults

In [ ]:
dfresults = dfresults.columns['10400101001']
dfresults
b= dfunc.columns
b

In [ ]:
dfunc.sort_values(dfunc.columns, axis=1, ascending=False)

In [ ]:
dfresults.head()

In [ ]:
column01 =[]
for column in pdr:
        str(column01.append(column))
       # n= df.sort_values(dfunc.columns[0],ascending=False)
    #return n
column01

In [ ]:
columns =[]
for column in dfunc:
        columns.append(column)
       # n= df.sort_values(dfunc.columns[0],ascending=False)
    #return n
columns
stuff = pd.DataFrame() #crea un nuevo dataframe

def keep(x):
    for e in columns:
         stuff[e] = dfunc[e] #empareja la nueva columna con la columna del df
    return stuff

keep(dfunc) 

In [ ]:
dforder.iloc[0:5,0:1].values

In [ ]:
for n in top5:
    m = dforder.iloc[0:5,0:n].index
    i = dforder.iloc[0:5,0:n].values
    print m,i

In [ ]:
def order (df):
    for column in df:
        c= str(columns)
        #print column
        n= df.sort_values(dfunc.columns[0],ascending=False)
    return n
order(dfunc)

In [ ]:
for name, values in dfunc.iteritems():
    print '{name}: {value}'.format(name=name, value=values[0])

In [ ]:
dfunc.apply(order, axis=1)

In [ ]:
dfunc= dfunc.iloc[:,:5]
#for column in dfunc:
 #   print(dfunc[column].sort_values(dfunc[column],ascending=False))
dfunc.tail()

In [ ]:
for column in dfunc:
    #a = dfunc.columns[column]
    d =dfunc.sort_values(dfunc.columns[0],ascending=False)
    #print(dfunc[column])
    print column,d
    
#dfunc.sort_values(dfunc.columns[1],ascending=False)

In [ ]:
def orfer 
group_a = dfunc.groupby(0)
group_a
for nombre, datos in group_a:
    print nombre
    print datos

In [ ]:
#Import data temporary only for test mode
#Madrid Data

dfmad = pd.read_csv('data/C2011_ccaa13_Indicadores_con_ID.csv', sep=';')
dfmad = dfmad.fillna(0)
indexed_df_mad = dfmad.set_index(['ID'])
#indexed_df_mad
X_normalized_indexed_df_mad = preprocessing.normalize(indexed_df_mad, norm='l1')
X_normalizeddf_madL2 = preprocessing.normalize(indexed_df_mad, norm='l2')
ix =indexed_df_mad.index
col = indexed_df_mad.columns
df__normalized_indexed_mad=pd.DataFrame(X_normalized_indexed_df_mad, index=[ix] ,columns=[col])
df__normalized_indexed_mad
#dfmad = dfmad.iloc[:,5:].fillna(0)
#Rest Data

In [ ]:
X_normalized_indexed_df_mad

In [ ]:
df01 = pd.read_csv('data/csv/C2011_ccaatotal_Indicadores_con_ID.csv', sep=';')
#dfmad.fillna(0)
df01 = df01.fillna(0)
indexed_df01 = df01.set_index(['ID'])
indexed_df01
X_normalized_indexed_df01 = preprocessing.normalize(indexed_df01, norm='l1')
X_normalizeddf01L2 = preprocessing.normalize(indexed_df01, norm='l2')
ix01 =indexed_df01.index
col01 = indexed_df01.columns
df__normalized_indexed01=pd.DataFrame(X_normalized_indexed_df01, index=[ix01] ,columns=[col01])
df__normalized_indexed01

In [ ]:
ix01 =indexed_df01.index
col01 = indexed_df01.columns
col01
ix01

In [ ]:
df__normalized_indexed01=pd.DataFrame(X_normalized_indexed_df01, index=[ix01] ,columns=[col01])


In [ ]:
#Processing data 
#Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
func =cosine_similarity(X_normalized_indexed_df_mad,df__normalized_indexed01)
dfunc = pd.DataFrame(func, index=[ix],columns=[ix01])
dfunc

In [ ]:
dforder = dfunc.sort_values(dfunc.columns[0],ascending=False)
ind = dforder.iloc[0:0, 0:1].columns
top5 = dforder.iloc[0:5,0:0].index
top5
#= pd.DataFrame(top5, index = ind),transpose

In [ ]:
dforder.iloc[0:0, 0:1].columns


In [ ]:
t= dfunc.transpose
t
#t = dfunc.sort_values(dfunc.columns[0],ascending=False).head(5)
#dfunc.transpose

In [ ]:
for index, row in dfunc.iterrows():
    print row.sort_values(dfunc.columns['row'],ascending=False)

In [ ]:
dforder({key: sorted(value.values(), reverse=True) \
    for key, value in dforder.to_dict().iteritems()})

In [ ]:
q = dforder.iterrows()

In [ ]:
def results(df):
    d = []
    for n in df.columns:
         d.append(df.sort_values(df.columns[n],ascending=False))
    return  d
   
#for col in mandforder.columns:

mandforder.sort_values(mandforder.columns[10],ascending=False)

In [ ]:
#Normalize all data
from sklearn import preprocessing
from sklearn.preprocessing import normalize
X_normalized_indexed_df01 = preprocessing.normalize(indexed_df01, norm='l1')
X_normalizeddf01L2 = preprocessing.normalize(df01, norm='l2')

In [ ]:
df01

In [ ]:
#Euclidean Distance
from sklearn.metrics.pairwise import euclidean_distances
eucl_dist= euclidean_distances(X_normalized_indexed_df01, df__normalized_indexed)
dfeu = pd.DataFrame(eucl_dist, index=[ix01],columns=[ix])
dfeu

In [ ]:
#Processing data 
#Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
func =cosine_similarity(X_normalized_indexed_df01, df__normalized_indexed)
dfunc = pd.DataFrame(func, index=[ix01],columns=[ix])
dfunc

In [ ]:
#Observed Data
dfmad.sample(10)
dfmad.shape
dfmad.head()

In [ ]:
df = dfmad.copy()
indexed_df = df.set_index(['ID'])

In [ ]:
indexed_df

In [ ]:
ix =indexed_df.index
ix1 = ix[0:20]

In [ ]:
col = indexed_df.columns
col

In [ ]:
indexed_df.head(10)

In [ ]:
#Normalize all data
from sklearn import preprocessing
from sklearn.preprocessing import normalize
X_normalized_indexed_df = preprocessing.normalize(indexed_df, norm='l1')
X_normalizedL2mad = preprocessing.normalize(dfmad, norm='l2')

In [ ]:
df__normalized_indexed=pd.DataFrame(X_normalized_indexed_df, index=[ix] ,columns=[col])
df__normalized_indexed

In [ ]:
df__normalized_indexed.columns = [col]

In [ ]:
df__normalized_indexed.head(10)

In [ ]:
#Processing data 
#Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
func =cosine_similarity(df__normalized_indexed[0:20], df__normalized_indexed)
dfunc = pd.DataFrame(func, index=[ix1],columns=[ix])
dfunc

In [ ]:
#Sort Results
dforder = dfunc.sort_values(dfunc.columns[0],ascending=False)
dforder.iloc[1:,[0]]


In [ ]:
def order_results(df):
    for i in df.columns:
        dfunc.columns[i].values
        #print df.sort_values(dfunc.columns[i],ascending=False)
        print df.columns
        
order_results(dfunc)

In [ ]:
#Processing data 
#Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
func =cosine_similarity(df__normalized_indexed[0:10], df__normalized_indexed)
#Euclidean Distance
from sklearn.metrics.pairwise import euclidean_distances
eucl_dist= euclidean_distances(X_normalizedmad[0:10], X_normalizedmad)
#Manhattan Distance
from sklearn.metrics.pairwise import manhattan_distances
manhattan_dist =  manhattan_distances(X_normalizedmad[0:10], X_normalizedmad)
#Minkowski distance

In [ ]:
%time
dfunc = pd.DataFrame(func, index=[ix1],columns=[ix])
#dfeucl_dist= pd.DataFrame(eucl_dist)
dfunc

In [ ]:
dfunc[:1]

In [ ]:
f= lambda x:x.max()
dfunc[:1].apply(f)

In [ ]:
%%time
dfeucl_dist

In [ ]:
#%%time
dfmanhattan_dist= pd.DataFrame(manhattan_dist)
dfmanhattan_dist

In [ ]:
#Order data & preparing to create table BBDD

In [ ]:
from IPython.display import Image
#Image("./data/cityData.png")

# <p style="text-align: center;">DATABASE SCHEMA</p>

![title](./data/cityData1.png)

In [ ]:
a = pd.DataFrame(indexed_df.isnull().sum())

In [ ]:
a.transpose()

In [ ]:
#C2011_Indicadores.csv
df01 = pd.read_csv('data/csv/C2011_Indicadores.csv', sep=';')
df01 = df01.set_index(['idsecc'])
df01= df01.iloc[:,5:]
#indexed_df01 = df01.set_index(['idsecc'])
df01

In [ ]:
df01 = pd.read_csv('data/csv/C2011_Indicadores.csv', sep=';')

df01= df.iloc[1:,[0]]
df01.shape
#dfmad.fillna(0)
df01 = df01.fillna(0)
indexed_df01 = df01.set_index(['idsecc'])
indexed_df01
X_normalized_indexed_df01 = preprocessing.normalize(indexed_df01, norm='l1')
X_normalizeddf01L2 = preprocessing.normalize(indexed_df01, norm='l2')
ix01 =indexed_df01.index
col01 = indexed_df01.columns
df__normalized_indexed01=pd.DataFrame(X_normalized_indexed_df01, index=[ix01] ,columns=[col01])
df__normalized_indexed01